# Submitting jobs to Triton 
## Gamma-optinar September 2023

The example problem we use here is a very simple algorithm for estimating the value of $\pi$. We know that the area of a circle is $\pi r^2$. If we consider a circle centered at the origin with a radius $r=1$, the area is $\pi$. To make thing simple, we only consider the positive orthant and thus, only a quarter of the circle, with an area of $\frac{\pi}{4}$. If we now sample points $(x,y)$ with both variables uniformly distributed between 0 and 1 (the default behavior of ```rand()``` ), and denote the fraction of points within the circle with $p$, we should have $p = \frac{A_{circle}}{A_{square}} = \frac{\pi}{4}$ or $\pi = 4p$. 

This is a notebook with the Bash kernel. That means this notebook is basically the standard Linux command line interface. One of the prerequisites for using Triton is basic Linux shell knowledge, which can be obtained, e.g., from the CodeRefinery [website](https://hands-on.coderefinery.org/C/). We can start by checking that we are in the correct directory:

In [ ]:
pwd

Great, now we load the Julia module so that we can use Julia-

In [ ]:
module load julia

Next, we will run a single Julia file using ```srun```:

In [ ]:
srun --cpus-per-task=1 --mem=1G --time=0:01:00 julia code/pi_montecarlo.jl

The problem with ```srun``` is that it blocks your shell, that is, you can't do anything else while it is running. If your script takes long to run, a better solution would be serial jobs using ```sbatch```. Let's take a look at the script and run it.

In [ ]:
cat code/script_pi.sh

In [ ]:
sbatch code/script_pi.sh

This way, your Julia file is run in the background (it might actually run on a different node) and you can continue working on other stuff. We also specified a file where the output of the file should go to. You can view the jobs in your queue using ```slurm queue```:

In [ ]:
slurm queue

Finally, you might start thinking that it's not a good idea to have just one estimate of the value of $\pi$. Maybe the method we used converges to different values. Maybe we want to know how fast it converges. To answer these concerns, we can run 50 simulations at the same time using _array jobs_. An array script looks almost identical to the previous script, except that it has a parameter ```array``` specifying how many instances we want to solve. The ```$SLURM_ARRAY_TASK_ID``` gives the array task ID that we then pass to the Julia file so that it knows the ID. This is convenient for writing the results to different files.

In [ ]:
cat code/array_script_pi.sh

In [ ]:
sbatch code/array_script_pi.sh

We can check that this indeed results in 50 jobs in the queue, all running in parallel.

In [ ]:
slurm queue

For more interesting stuff (and a familiar example), check out the [Triton documentation](https://scicomp.aalto.fi/triton/#).